卷积操作得到了空间信息

卷积部分叫做特征提取,全连接部分叫做分类器

光敏电阻,变成光照强度,然后组成图片,黑白的,
光敏器件,红色传感器,绿色传感器,蓝色传感器,RGB


灰度级别,转换成0-255,转换成图像
插值
矢量图像---人工生成,圆心,边,直径,描述,放大不失真
栅格图像---小格子,放大失真
    Input Channel Width,Height


卷积运算
Batch * N*W*H

In [2]:
import torch

In [3]:
in_channels,out_channels=5,10
width,height=100,100
kernel_size=3
batch_size=1

In [4]:
input=torch.randn(batch_size,
                  in_channels,
                  width,
                  height)

In [5]:
conv_layer=torch.nn.Conv2d(in_channels,
                          out_channels,
                          kernel_size=kernel_size)

In [6]:
output=conv_layer(input)

In [7]:
print(input.shape)

torch.Size([1, 5, 100, 100])


In [8]:
print(output.shape)

torch.Size([1, 10, 98, 98])


In [10]:
print(conv_layer.weight.shape)

torch.Size([10, 5, 3, 3])


padding:kernel/2----整除向下取整

In [11]:
import torch

In [12]:
input=[3,4,6,5,7,
       2,4,6,8,2,
       1,6,7,8,4,
       9,7,4,6,2,
       3,7,5,4,1]

In [14]:
input=torch.Tensor(input).view(1,1,5,5)
conv_layer=torch.nn.Conv2d(1,1,kernel_size=3,padding=1,bias=False)

In [15]:
kernel=torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3)

In [16]:
conv_layer.weight.data=kernel.data

In [17]:
output=conv_layer(input)

In [18]:
print(output)

tensor([[[[ 91., 168., 224., 215., 127.],
          [114., 211., 295., 262., 149.],
          [192., 259., 282., 214., 122.],
          [194., 251., 253., 169.,  86.],
          [ 96., 112., 110.,  68.,  31.]]]], grad_fn=<ThnnConv2DBackward>)


In [19]:
import torch
input=[3,4,6,5,
       2,4,6,8,
       1,6,7,8,
       9,7,4,6,
      ]

In [20]:
input=torch.Tensor(input).view(1,1,4,4)

In [21]:
maxpooling_layer=torch.nn.MaxPool2d(kernel_size=2)

In [22]:
output=maxpooling_layer(input)

In [23]:
print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2=torch.nn.Conv2d(10,20,kernel_size=5)
        self.pooling=torch.nn.MaxPool2d(2)
        self.fc=torch.nn.Linear(320,10)
    def forward(self,x):
        batch_size=x.size(0)
        x=F.relu(self.pooling(self.conv1(x)))
        x=F.relu(self.pooling(self.conv2(x)))
        x=x.view(batch_size,-1)
        x=self.fc(x)
        return x
model=Net()

In [28]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2=torch.nn.Conv2d(10,20,kernel_size=5)
        self.pooling=torch.nn.MaxPool2d(2)
        self.fc=torch.nn.Linear(320,10)
    def forward(self,x):
        batch_size=x.size(0)
        x=F.relu(self.pooling(self.conv1(x)))
        x=F.relu(self.pooling(self.conv2(x)))
        x=x.view(batch_size,-1)
        x=self.fc(x)
        return x
model=Net()

In [30]:
#define device as the first visible cuda device if we have CUDA avalible
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)#convert  parameters and buffers of all modules to CUDA Tensor

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

In [33]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs, target = data
        #send the inputs and targets at every step to the GPU
        inputs,target=inputs.to(device),target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0        

In [32]:
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images,labels=images.to(device),labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%'%(100*correct/total))